# REST Inference

## Setup

Verify that following variable settings match your deployed model's resource name and rest URL. The following code assumes that the kube service is in the same namespace, but you could refer to it in full with the namespace, for example: `http://modelmesh-serving.project-name.svc.cluster.local:8008/v2/models/fraud/infer`

In [40]:
infer_url = "https://caloricexp-neeraj-jain-dev.apps.sandbox-m3.1530.p1.openshiftapps.com/v2/models/caloricexp/infer"

## Request Function

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [47]:
import requests

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "keras_tensor",
                "shape": [1, 35],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [42]:
#Load the scaler
import pickle
with open('artifact/column_transformer.pkl', 'rb') as handle:
    ct = pickle.load(handle)

In [43]:
import pandas as pd

# Define your input data as a list
data = pd.read_csv('X_test.csv')
data

,day,age,gender,weight,height_in_cms,digestive_disorder,type_of_workout,additional_activity,total_calories_burned,body_temperature,workout_intensity,followed_diet_plan,taking_supplements,fitness_goal,target_weight,target_time_in_months,actual_calorie_intake_in_kcal_per_day,actual_carb_intake_in_gms,actual_protein_intake_in_gms,actual_fat_intake_in_gms
0,Friday,42,Male,77,155,No,Light Weight Training,Cardio,802,38,Medium,Yes,No,Wellness,0,0,2320,290,116,77
1,Saturday,38,Male,41,174,No,Calesthenics,Aerobics,901,38,Low,No,No,Weight Gain,39,13,1477,184,73,49
2,Friday,48,Male,66,147,No,Calesthenics,Aerobics,734,37,Low,No,No,Wellness,0,0,1682,210,84,56
3,Wednesday,49,Male,85,179,Yes,Light Weight Training,Yoga,351,39,Medium,Yes,No,Weight Loss,5,1,2095,261,104,69
4,Wednesday,42,Male,84,157,No,Calesthenics,Cardio,710,39,Low,No,No,Weight Loss,4,1,1860,232,93,62


In [44]:
# Transform the DataFrame using the column transformer
transformed_data = ct.transform(data.sample(1))
transformed_data

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.86828708,
         0.70668537, -0.07824578,  0.93619311,  1.21802627, -0.7380973 ,
        -0.77501692, -0.95721982, -0.95862794, -0.93051376, -0.91543942]])

In [45]:
transformed_data.shape

(1, 35)

In [48]:
prediction = rest_request(transformed_data.tolist())
prediction

[1773.8453]